In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None  # default='warn' surpress chain assignment warning

from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

#import nltk
#nltk.download('all')

import numpy as np

# import re
# import string
# from nltk.corpus import wordnet
# from nltk.corpus import brown
# from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
# from nltk.tokenize import TweetTokenizer
# from nltk.stem import WordNetLemmatizer
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix, hstack

#from textblob import Word

#from collections import Counter

#import gensim

from numpy import savetxt
import numpy as np
import scipy.sparse

# XGBoost
import xgboost as xgb
from xgboost import XGBClassifier

# sklearn
from sklearn import model_selection
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.compose import ColumnTransformer

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
import seaborn as sns

#import torch

# turn off userwarnings
import warnings
warnings.filterwarnings('ignore')

#pip install tokenization

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from scipy.sparse import load_npz

# Paths to features from each extraction method
# NepalEQ_hashtag_arr_matrix_path = '/content/drive/MyDrive/NepalEQ_hashtag_arr_matrix.npy'
QueenslandFLD_hashtag_arr_matrix_path = '/content/drive/MyDrive/QueenslandFLD_hashtag_arr_matrix.npy'
QueenslandFLD_hashtag_lda_arr_matrix_path = '/content/drive/MyDrive/QueenslandFLD_hashtag_lda_arr_matrix.npy'
QueenslandFLD_hashtag_cooccurrence_arr_matrix_path = '/content/drive/MyDrive/QueenslandFLD_hashtag_cooccurrence_arr_matrix.npy'
QueenslandFLD_hashtag_embeddings_arr_matrix_path = '/content/drive/MyDrive/QueenslandFLD_hashtag_embeddings_arr_matrix.npy'
QueenslandFLD_hashtag_tfidf_arr_matrix_path = '/content/drive/MyDrive/QueenslandFLD_hashtag_tfidf_arr_matrix.npy'
# NepalEQ_retweet_arr_matrix_path = '/content/drive/MyDrive/NepalEQ_retweet_arr_matrix.npy'
# QueenslandFLD_retweet_arr_matrix_path = '/content/drive/MyDrive/QueenslandFLD_retweet_arr_matrix.npy'
# NepalEQ_df_em_1gram_matrix_path = '/content/drive/MyDrive/NepalEQ_df_em_1gram_matrix.npy'
# QueenslandFLD_df_em_1gram_matrix_path = '/content/drive/MyDrive/QueenslandFLD_df_em_1gram_matrix.npy'
# NepalEQ_POS_matrix_path = '/content/drive/MyDrive/NepalEQ_POS_matrix.npy'
# QueenslandFLD_POS_matrix_path = '/content/drive/MyDrive/QueenslandFLD_POS_matrix.npy'
# NepalEQ_df_embeddings_fasttext_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_fasttext_matrix.npy'
# QueenslandFLD_df_embeddings_fasttext_matrix_path = '/content/drive/MyDrive/QueenslandFLD_df_embeddings_fasttext_matrix.npy'
# NepalEQ_df_embeddings_word2vec_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_word2vec_matrix.npy'
# QueenslandFLD_df_embeddings_word2vec_matrix_path = '/content/drive/MyDrive/QueenslandFLD_df_embeddings_word2vec_matrix.npy'
# NepalEQ_df_embeddings_glove_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_glove_matrix.npy'
# QueenslandFLD_df_embeddings_glove_matrix_path = '/content/drive/MyDrive/QueenslandFLD_df_embeddings_glove_matrix.npy'
# NepalEQ_df_bert_input_matrix_path = '/content/drive/MyDrive/NepalEQ_df_bert_input_matrix.npy'
# QueenslandFLD_df_bert_input_matrix_path = '/content/drive/MyDrive/QueenslandFLD_df_bert_input_matrix.npy'
# NepalEQ_df_tfidf_1gram_matrix_path = '/content/drive/MyDrive/NepalEQ_df_tfidf_1gram_matrix.npy'
# QueenslandFLD_df_tfidf_1gram_matrix_path = '/content/drive/MyDrive/QueenslandFLD_df_tfidf_1gram_matrix.npy'
# NepalEQ_TFIDF_HT_RT_POS_FT_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_FT_hstack.npy'
# QueenslandFLD_TFIDF_HT_RT_POS_FT_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_RT_POS_FT_hstack.npy'
# NepalEQ_TFIDF_HT_RT_POS_w2v_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_w2v_hstack.npy'
# QueenslandFLD_TFIDF_HT_RT_POS_w2v_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_RT_POS_w2v_hstack.npy'
# NepalEQ_TFIDF_HT_RT_POS_glove_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_glove_hstack.npy'
# QueenslandFLD_TFIDF_HT_RT_POS_glove_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_RT_POS_glove_hstack.npy'
# NepalEQ_TFIDF_HT_RT_POS_bert_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_bert_hstack.npy'
# QueenslandFLD_TFIDF_HT_RT_POS_bert_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_RT_POS_bert_hstack.npy'
# NepalEQ_TFIDF_HT_POS_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_POS_hstack.npy'
# QueenslandFLD_TFIDF_HT_POS_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_POS_hstack.npy'
# QueenslandFLD_TFIDF_HT_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_hstack.npy'
# QueenslandFLD_HT_POS_hstack_path = '/content/drive/MyDrive/QueenslandFLD_HT_POS_hstack.npy'
# QueenslandFLD_HT_FT_hstack_path = '/content/drive/MyDrive/QueenslandFLD_HT_FT_hstack.npy'
# QueenslandFLD_HT_glove_hstack_path = '/content/drive/MyDrive/QueenslandFLD_HT_glove_hstack.npy'
# QueenslandFLD_HT_w2v_hstack_path = '/content/drive/MyDrive/QueenslandFLD_HT_w2v_hstack.npy'
# QueenslandFLD_HT_bert_hstack_path = '/content/drive/MyDrive/QueenslandFLD_HT_bert_hstack.npy'
# QueenslandFLD_POS_FT_hstack_path = '/content/drive/MyDrive/QueenslandFLD_POS_FT_hstack.npy'
# QueenslandFLD_POS_glove_hstack_path = '/content/drive/MyDrive/QueenslandFLD_POS_glove_hstack.npy'
# QueenslandFLD_POS_w2v_hstack_path = '/content/drive/MyDrive/QueenslandFLD_POS_w2v_hstack.npy'
# QueenslandFLD_POS_bert_hstack_path = '/content/drive/MyDrive/QueenslandFLD_POS_bert_hstack.npy'
# QueenslandFLD_TFIDF_POS_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_POS_hstack.npy'
# QueenslandFLD_TFIDF_FT_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_FT_hstack.npy'
# QueenslandFLD_TFIDF_glove_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_glove_hstack.npy'
# QueenslandFLD_TFIDF_w2v_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_w2v_hstack.npy'
# QueenslandFLD_TFIDF_bert_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_bert_hstack.npy'
# QueenslandFLD_TFIDF_HT_POS_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_POS_hstack.npy'
# QueenslandFLD_TFIDF_HT_FT_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_FT_hstack.npy'
# QueenslandFLD_TFIDF_HT_glove_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_glove_hstack.npy'
# QueenslandFLD_TFIDF_HT_w2v_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_w2v_hstack.npy'
# QueenslandFLD_TFIDF_HT_bert_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_bert_hstack.npy'
# QueenslandFLD_TFIDF_HT_POS_FT_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_POS_FT_hstack.npy'
# QueenslandFLD_TFIDF_HT_POS_glove_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_POS_glove_hstack.npy'
# QueenslandFLD_TFIDF_HT_POS_w2v_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_POS_w2v_hstack.npy'
# QueenslandFLD_TFIDF_HT_POS_BERT_hstack_path = '/content/drive/MyDrive/QueenslandFLD_TFIDF_HT_POS_BERT_hstack.npy'




# Paths to traget variables
# NepalEQ_df_target_path = '/content/drive/MyDrive/NepalEQ_df_label.npy'
QueenslandFLD_df_target = '/content/drive/MyDrive/QueenslandFLD_df_label.npy'

# load features
# NepalEQ_hashtag = np.load(NepalEQ_hashtag_arr_matrix_path)
# QueenslandFLD_df_tfidf_1gram = np.load(QueenslandFLD_df_tfidf_1gram_matrix_path)
QueenslandFLD_hashtag = np.load(QueenslandFLD_hashtag_arr_matrix_path)
QueenslandFLD_hashtag_lda = np.load(QueenslandFLD_hashtag_lda_arr_matrix_path)
QueenslandFLD_hashtag_cooccurrence = np.load(QueenslandFLD_hashtag_cooccurrence_arr_matrix_path)
QueenslandFLD_hashtag_embeddings = np.load(QueenslandFLD_hashtag_embeddings_arr_matrix_path)
QueenslandFLD_hashtag_tfidf = np.load(QueenslandFLD_hashtag_tfidf_arr_matrix_path)
# NepalEQ_retweet = np.load(NepalEQ_retweet_arr_matrix_path)
# QueenslandFLD_retweet = np.load(QueenslandFLD_retweet_arr_matrix_path)
# NepalEQ_df_em_1gram = np.load(NepalEQ_df_em_1gram_matrix_path)
# QueenslandFLD_df_em_1gram = np.load(QueenslandFLD_df_em_1gram_matrix_path)
# NepalEQ_POS = np.load(NepalEQ_POS_matrix_path)
# QueenslandFLD_POS = np.load(QueenslandFLD_POS_matrix_path)
# NepalEQ_df_embeddings_fasttext = np.load(NepalEQ_df_embeddings_fasttext_matrix_path)
# QueenslandFLD_df_embeddings_fasttext = np.load(QueenslandFLD_df_embeddings_fasttext_matrix_path)
# QueenslandFLD_df_embeddings_glove = np.load(QueenslandFLD_df_embeddings_glove_matrix_path)
# NepalEQ_df_embeddings_word2vec = np.load(NepalEQ_df_embeddings_word2vec_matrix_path)
# QueenslandFLD_df_embeddings_word2vec = np.load(QueenslandFLD_df_embeddings_word2vec_matrix_path)
# NepalEQ_df_embeddings_glove = np.load(NepalEQ_df_embeddings_glove_matrix_path)
# NepalEQ_df_bert_input = np.load(NepalEQ_df_bert_input_matrix_path)
# QueenslandFLD_df_bert_input = np.load(QueenslandFLD_df_bert_input_matrix_path)
# NepalEQ_df_tfidf_1gram = np.load(NepalEQ_df_tfidf_1gram_matrix_path)
# NepalEQ_TFIDF_HT_RT_POS_FT = np.load(NepalEQ_TFIDF_HT_RT_POS_FT_hstack_path)
# QueenslandFLD_TFIDF_HT_RT_POS_FT = np.load(QueenslandFLD_TFIDF_HT_RT_POS_FT_hstack_path)
# NepalEQ_TFIDF_HT_RT_POS_w2v = np.load(NepalEQ_TFIDF_HT_RT_POS_w2v_hstack_path)
# QueenslandFLD_TFIDF_HT_RT_POS_w2v = np.load(QueenslandFLD_TFIDF_HT_RT_POS_w2v_hstack_path)
# NepalEQ_TFIDF_HT_RT_POS_glove = np.load(NepalEQ_TFIDF_HT_RT_POS_glove_hstack_path)
# QueenslandFLD_TFIDF_HT_RT_POS_glove = np.load(QueenslandFLD_TFIDF_HT_RT_POS_glove_hstack_path)
# NepalEQ_TFIDF_HT_RT_POS_bert = np.load(NepalEQ_TFIDF_HT_RT_POS_bert_hstack_path)
# QueenslandFLD_TFIDF_HT_RT_POS_bert = np.load(QueenslandFLD_TFIDF_HT_RT_POS_bert_hstack_path)
# NepalEQ_TFIDF_HT_POS = np.load(NepalEQ_TFIDF_HT_POS_hstack_path)
# QueenslandFLD_TFIDF_HT_POS = np.load(QueenslandFLD_TFIDF_HT_POS_hstack_path)
# QueenslandFLD_TFIDF_HT_hstack = np.load(QueenslandFLD_TFIDF_HT_hstack_path)
# QueenslandFLD_HT_POS_hstack = np.load(QueenslandFLD_HT_POS_hstack_path)
# QueenslandFLD_HT_FT_hstack = np.load(QueenslandFLD_HT_FT_hstack_path)
# QueenslandFLD_HT_glove_hstack = np.load(QueenslandFLD_HT_glove_hstack_path)
# QueenslandFLD_HT_w2v_hstack = np.load(QueenslandFLD_HT_w2v_hstack_path)
# QueenslandFLD_HT_bert_hstack = np.load(QueenslandFLD_HT_bert_hstack_path)
# QueenslandFLD_POS_FT_hstack = np.load(QueenslandFLD_POS_FT_hstack_path)
# QueenslandFLD_POS_glove_hstack = np.load(QueenslandFLD_POS_glove_hstack_path)
# QueenslandFLD_POS_w2v_hstack = np.load(QueenslandFLD_POS_w2v_hstack_path)
# QueenslandFLD_POS_bert_hstack = np.load(QueenslandFLD_POS_bert_hstack_path)
# QueenslandFLD_TFIDF_POS_hstack = np.load(QueenslandFLD_TFIDF_POS_hstack_path)
# QueenslandFLD_TFIDF_FT_hstack = np.load(QueenslandFLD_TFIDF_FT_hstack_path)
# QueenslandFLD_TFIDF_glove_hstack = np.load(QueenslandFLD_TFIDF_glove_hstack_path)
# QueenslandFLD_TFIDF_w2v_hstack = np.load(QueenslandFLD_TFIDF_w2v_hstack_path)
# QueenslandFLD_TFIDF_bert_hstack = np.load(QueenslandFLD_TFIDF_bert_hstack_path)
# QueenslandFLD_TFIDF_HT_POS_hstack = np.load(QueenslandFLD_TFIDF_HT_POS_hstack_path)
# QueenslandFLD_TFIDF_HT_FT_hstack = np.load(QueenslandFLD_TFIDF_HT_FT_hstack_path)
# QueenslandFLD_TFIDF_HT_glove_hstack = np.load(QueenslandFLD_TFIDF_HT_glove_hstack_path)
# QueenslandFLD_TFIDF_HT_w2v_hstack = np.load(QueenslandFLD_TFIDF_HT_w2v_hstack_path)
# QueenslandFLD_TFIDF_HT_bert_hstack = np.load(QueenslandFLD_TFIDF_HT_bert_hstack_path)
# QueenslandFLD_TFIDF_HT_POS_FT_hstack = np.load(QueenslandFLD_TFIDF_HT_POS_FT_hstack_path)
# QueenslandFLD_TFIDF_HT_POS_glove_hstack = np.load(QueenslandFLD_TFIDF_HT_POS_glove_hstack_path)
# QueenslandFLD_TFIDF_HT_POS_w2v_hstack = np.load(QueenslandFLD_TFIDF_HT_POS_w2v_hstack_path)
# QueenslandFLD_TFIDF_HT_POS_BERT_hstack = np.load(QueenslandFLD_TFIDF_HT_POS_BERT_hstack_path)


# load targets
# NepalEQ_df_target = np.load(NepalEQ_df_target_path)
QueenslandFLD_target = np.load(QueenslandFLD_df_target)


df_list = [QueenslandFLD_hashtag, QueenslandFLD_hashtag_embeddings, QueenslandFLD_hashtag_tfidf]

#, QueenslandFLD_POS, QueenslandFLD_df_embeddings_fasttext, QueenslandFLD_df_embeddings_word2vec,
          #  QueenslandFLD_df_embeddings_glove, QueenslandFLD_df_bert_input, QueenslandFLD_df_tfidf_1gram,
          #  QueenslandFLD_TFIDF_HT_hstack, QueenslandFLD_HT_POS_hstack, QueenslandFLD_HT_FT_hstack,
          #  QueenslandFLD_HT_glove_hstack, QueenslandFLD_HT_w2v_hstack, QueenslandFLD_HT_bert_hstack,
          #  QueenslandFLD_POS_FT_hstack, QueenslandFLD_POS_glove_hstack, QueenslandFLD_POS_w2v_hstack,
          #  QueenslandFLD_POS_bert_hstack, QueenslandFLD_TFIDF_POS_hstack, QueenslandFLD_TFIDF_FT_hstack,
          #  QueenslandFLD_TFIDF_glove_hstack, QueenslandFLD_TFIDF_w2v_hstack, QueenslandFLD_TFIDF_bert_hstack,
          #  QueenslandFLD_TFIDF_HT_POS_hstack, QueenslandFLD_TFIDF_HT_FT_hstack, QueenslandFLD_TFIDF_HT_glove_hstack, QueenslandFLD_TFIDF_HT_w2v_hstack,
          #  QueenslandFLD_TFIDF_HT_bert_hstack, QueenslandFLD_TFIDF_HT_POS_FT_hstack, QueenslandFLD_TFIDF_HT_POS_glove_hstack,
          #  QueenslandFLD_TFIDF_HT_POS_w2v_hstack, QueenslandFLD_TFIDF_HT_POS_BERT_hstack]
df_names = ['QueenslandFLD_hashtag', 'QueenslandFLD_hashtag_embeddings', 'QueenslandFLD_hashtag_tfidf']

#, 'QueenslandFLD_POS', 'QueenslandFLD_df_embeddings_fasttext', 'QueenslandFLD_df_embeddings_word2vec',
          #  'QueenslandFLD_df_embeddings_glove', 'QueenslandFLD_df_bert_input', 'QueenslandFLD_df_tfidf_1gram',
          #  'QueenslandFLD_TFIDF_HT_hstack', 'QueenslandFLD_HT_POS_hstack', 'QueenslandFLD_HT_FT_hstack',
          #  'QueenslandFLD_HT_glove_hstack', 'QueenslandFLD_HT_w2v_hstack', 'QueenslandFLD_HT_bert_hstack',
          #  'QueenslandFLD_POS_FT_hstack', 'QueenslandFLD_POS_glove_hstack', 'QueenslandFLD_POS_w2v_hstack',
          #  'QueenslandFLD_POS_bert_hstack', 'QueenslandFLD_TFIDF_POS_hstack', 'QueenslandFLD_TFIDF_FT_hstack',
          #  'QueenslandFLD_TFIDF_glove_hstack', 'QueenslandFLD_TFIDF_w2v_hstack', 'QueenslandFLD_TFIDF_bert_hstack',
          #  'QueenslandFLD_TFIDF_HT_POS_hstack', 'QueenslandFLD_TFIDF_HT_FT_hstack', 'QueenslandFLD_TFIDF_HT_glove_hstack', 'QueenslandFLD_TFIDF_HT_w2v_hstack',
          #  'QueenslandFLD_TFIDF_HT_bert_hstack', 'QueenslandFLD_TFIDF_HT_POS_FT_hstack', 'QueenslandFLD_TFIDF_HT_POS_glove_hstack',
          #  'QueenslandFLD_TFIDF_HT_POS_w2v_hstack', 'QueenslandFLD_TFIDF_HT_POS_BERT_hstack']

In [3]:
# from scipy.sparse import load_npz

# # Paths to features from each extraction method
# # NepalEQ_hashtag_arr_matrix_path = '/content/drive/MyDrive/NepalEQ_hashtag_arr_matrix.npy'
# NepalEQ_hashtag_arr_matrix_path = '/content/drive/MyDrive/NepalEQ_hashtag_arr_matrix.npy'
# # NepalEQ_retweet_arr_matrix_path = '/content/drive/MyDrive/NepalEQ_retweet_arr_matrix.npy'
# # NepalEQ_retweet_arr_matrix_path = '/content/drive/MyDrive/NepalEQ_retweet_arr_matrix.npy'
# # NepalEQ_df_em_1gram_matrix_path = '/content/drive/MyDrive/NepalEQ_df_em_1gram_matrix.npy'
# # NepalEQ_df_em_1gram_matrix_path = '/content/drive/MyDrive/NepalEQ_df_em_1gram_matrix.npy'
# # NepalEQ_POS_matrix_path = '/content/drive/MyDrive/NepalEQ_POS_matrix.npy'
# NepalEQ_POS_matrix_path = '/content/drive/MyDrive/NepalEQ_POS_matrix.npy'
# # NepalEQ_df_embeddings_fasttext_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_fasttext_matrix.npy'
# NepalEQ_df_embeddings_fasttext_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_fasttext_matrix.npy'
# # NepalEQ_df_embeddings_word2vec_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_word2vec_matrix.npy'
# NepalEQ_df_embeddings_word2vec_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_word2vec_matrix.npy'
# # NepalEQ_df_embeddings_glove_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_glove_matrix.npy'
# NepalEQ_df_embeddings_glove_matrix_path = '/content/drive/MyDrive/NepalEQ_df_embeddings_glove_matrix.npy'
# # NepalEQ_df_bert_input_matrix_path = '/content/drive/MyDrive/NepalEQ_df_bert_input_matrix.npy'
# NepalEQ_df_bert_input_matrix_path = '/content/drive/MyDrive/NepalEQ_df_bert_input_matrix.npy'
# # NepalEQ_df_tfidf_1gram_matrix_path = '/content/drive/MyDrive/NepalEQ_df_tfidf_1gram_matrix.npy'
# NepalEQ_df_tfidf_1gram_matrix_path = '/content/drive/MyDrive/NepalEQ_df_tfidf_1gram_matrix.npy'
# # NepalEQ_TFIDF_HT_RT_POS_FT_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_FT_hstack.npy'
# # NepalEQ_TFIDF_HT_RT_POS_FT_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_FT_hstack.npy'
# # NepalEQ_TFIDF_HT_RT_POS_w2v_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_w2v_hstack.npy'
# # NepalEQ_TFIDF_HT_RT_POS_w2v_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_w2v_hstack.npy'
# # NepalEQ_TFIDF_HT_RT_POS_glove_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_glove_hstack.npy'
# # NepalEQ_TFIDF_HT_RT_POS_glove_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_glove_hstack.npy'
# # NepalEQ_TFIDF_HT_RT_POS_bert_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_bert_hstack.npy'
# # NepalEQ_TFIDF_HT_RT_POS_bert_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_RT_POS_bert_hstack.npy'
# # NepalEQ_TFIDF_HT_POS_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_POS_hstack.npy'
# # NepalEQ_TFIDF_HT_POS_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_POS_hstack.npy'
# NepalEQ_TFIDF_HT_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_hstack.npy'
# NepalEQ_HT_POS_hstack_path = '/content/drive/MyDrive/NepalEQ_HT_POS_hstack.npy'
# NepalEQ_HT_FT_hstack_path = '/content/drive/MyDrive/NepalEQ_HT_FT_hstack.npy'
# NepalEQ_HT_glove_hstack_path = '/content/drive/MyDrive/NepalEQ_HT_glove_hstack.npy'
# NepalEQ_HT_w2v_hstack_path = '/content/drive/MyDrive/NepalEQ_HT_w2v_hstack.npy'
# NepalEQ_HT_bert_hstack_path = '/content/drive/MyDrive/NepalEQ_HT_bert_hstack.npy'
# NepalEQ_POS_FT_hstack_path = '/content/drive/MyDrive/NepalEQ_POS_FT_hstack.npy'
# NepalEQ_POS_glove_hstack_path = '/content/drive/MyDrive/NepalEQ_POS_glove_hstack.npy'
# NepalEQ_POS_w2v_hstack_path = '/content/drive/MyDrive/NepalEQ_POS_w2v_hstack.npy'
# NepalEQ_POS_bert_hstack_path = '/content/drive/MyDrive/NepalEQ_POS_bert_hstackk.npy'
# NepalEQ_TFIDF_POS_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_POS_hstack.npy'
# NepalEQ_TFIDF_FT_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_FT_hstack.npy'
# NepalEQ_TFIDF_glove_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_glove_hstack.npy'
# NepalEQ_TFIDF_w2v_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_w2v_hstack.npy'
# NepalEQ_TFIDF_bert_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_bert_hstack.npy'
# NepalEQ_TFIDF_HT_POS_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_POS_hstack.npy'
# NepalEQ_TFIDF_HT_FT_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_FT_hstack.npy'
# NepalEQ_TFIDF_HT_glove_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_glove_hstack.npy'
# NepalEQ_TFIDF_HT_w2v_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_w2v_hstack.npy'
# NepalEQ_TFIDF_HT_bert_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_bert_hstack.npy'
# NepalEQ_TFIDF_HT_POS_FT_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_POS_FT_hstack.npy'
# NepalEQ_TFIDF_HT_POS_glove_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_POS_glove_hstack.npy'
# NepalEQ_TFIDF_HT_POS_w2v_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_POS_w2v_hstack.npy'
# NepalEQ_TFIDF_HT_POS_BERT_hstack_path = '/content/drive/MyDrive/NepalEQ_TFIDF_HT_POS_BERT_hstack.npy'




# # Paths to traget variables
# NepalEQ_df_target_path = '/content/drive/MyDrive/NepalEQ_df_label.npy'
# NepalEQ_df_target = '/content/drive/MyDrive/NepalEQ_df_label.npy'

# # load features



# # NepalEQ_hashtag = np.load(NepalEQ_hashtag_arr_matrix_path)
# # NepalEQ_retweet = np.load(NepalEQ_retweet_arr_matrix_path)
# # NepalEQ_retweet = np.load(NepalEQ_retweet_arr_matrix_path)
# # NepalEQ_df_em_1gram = np.load(NepalEQ_df_em_1gram_matrix_path)
# # NepalEQ_df_em_1gram = np.load(NepalEQ_df_em_1gram_matrix_path)
# # NepalEQ_POS = np.load(NepalEQ_POS_matrix_path)
# # NepalEQ_df_embeddings_fasttext = np.load(NepalEQ_df_embeddings_fasttext_matrix_path)
# # NepalEQ_df_embeddings_word2vec = np.load(NepalEQ_df_embeddings_word2vec_matrix_path)
# # NepalEQ_df_embeddings_glove = np.load(NepalEQ_df_embeddings_glove_matrix_path)
# # NepalEQ_df_bert_input = np.load(NepalEQ_df_bert_input_matrix_path)
# # NepalEQ_df_tfidf_1gram = np.load(NepalEQ_df_tfidf_1gram_matrix_path)
# # NepalEQ_TFIDF_HT_RT_POS_FT = np.load(NepalEQ_TFIDF_HT_RT_POS_FT_hstack_path)
# # NepalEQ_TFIDF_HT_RT_POS_FT = np.load(NepalEQ_TFIDF_HT_RT_POS_FT_hstack_path)
# # NepalEQ_TFIDF_HT_RT_POS_w2v = np.load(NepalEQ_TFIDF_HT_RT_POS_w2v_hstack_path)
# # NepalEQ_TFIDF_HT_RT_POS_w2v = np.load(NepalEQ_TFIDF_HT_RT_POS_w2v_hstack_path)
# # NepalEQ_TFIDF_HT_RT_POS_glove = np.load(NepalEQ_TFIDF_HT_RT_POS_glove_hstack_path)
# # NepalEQ_TFIDF_HT_RT_POS_glove = np.load(NepalEQ_TFIDF_HT_RT_POS_glove_hstack_path)
# # NepalEQ_TFIDF_HT_RT_POS_bert = np.load(NepalEQ_TFIDF_HT_RT_POS_bert_hstack_path)
# # NepalEQ_TFIDF_HT_RT_POS_bert = np.load(NepalEQ_TFIDF_HT_RT_POS_bert_hstack_path)
# # NepalEQ_TFIDF_HT_POS = np.load(NepalEQ_TFIDF_HT_POS_hstack_path)
# # NepalEQ_TFIDF_HT_POS = np.load(NepalEQ_TFIDF_HT_POS_hstack_path)


# # NepalEQ_hashtag = np.load(NepalEQ_hashtag_arr_matrix_path)
# # NepalEQ_POS = np.load(NepalEQ_POS_matrix_path)
# # NepalEQ_df_embeddings_fasttext = np.load(NepalEQ_df_embeddings_fasttext_matrix_path)
# # NepalEQ_df_embeddings_word2vec = np.load(NepalEQ_df_embeddings_word2vec_matrix_path)
# # NepalEQ_df_embeddings_glove = np.load(NepalEQ_df_embeddings_glove_matrix_path)
# # NepalEQ_df_bert_input = np.load(NepalEQ_df_bert_input_matrix_path)
# # NepalEQ_df_tfidf_1gram = np.load(NepalEQ_df_tfidf_1gram_matrix_path)
# # NepalEQ_TFIDF_HT_hstack = np.load(NepalEQ_TFIDF_HT_hstack_path)
# # NepalEQ_HT_POS_hstack = np.load(NepalEQ_HT_POS_hstack_path)
# # NepalEQ_HT_FT_hstack = np.load(NepalEQ_HT_FT_hstack_path)
# # NepalEQ_HT_glove_hstack = np.load(NepalEQ_HT_glove_hstack_path)
# # NepalEQ_HT_w2v_hstack = np.load(NepalEQ_HT_w2v_hstack_path)
# # NepalEQ_HT_bert_hstack = np.load(NepalEQ_HT_bert_hstack_path)
# # NepalEQ_POS_FT_hstack = np.load(NepalEQ_POS_FT_hstack_path)
# # NepalEQ_POS_glove_hstack = np.load(NepalEQ_POS_glove_hstack_path)
# # NepalEQ_POS_w2v_hstack = np.load(NepalEQ_POS_w2v_hstack_path)
# # NepalEQ_POS_bert_hstack = np.load(NepalEQ_POS_bert_hstack_path)
# # NepalEQ_TFIDF_POS_hstack = np.load(NepalEQ_TFIDF_POS_hstack_path)
# # NepalEQ_TFIDF_FT_hstack = np.load(NepalEQ_TFIDF_FT_hstack_path)
# # NepalEQ_TFIDF_glove_hstack = np.load(NepalEQ_TFIDF_glove_hstack_path)
# # NepalEQ_TFIDF_w2v_hstack = np.load(NepalEQ_TFIDF_w2v_hstack_path)
# # NepalEQ_TFIDF_bert_hstack = np.load(NepalEQ_TFIDF_bert_hstack_path)
# # NepalEQ_TFIDF_HT_POS_hstack = np.load(NepalEQ_TFIDF_HT_POS_hstack_path)
# # NepalEQ_TFIDF_HT_FT_hstack = np.load(NepalEQ_TFIDF_HT_FT_hstack_path)
# # NepalEQ_TFIDF_HT_glove_hstack = np.load(NepalEQ_TFIDF_HT_glove_hstack_path)
# # NepalEQ_TFIDF_HT_w2v_hstack = np.load(NepalEQ_TFIDF_HT_w2v_hstack_path)

# NepalEQ_TFIDF_HT_bert_hstack = np.load(NepalEQ_TFIDF_HT_bert_hstack_path)
# NepalEQ_TFIDF_HT_POS_FT_hstack = np.load(NepalEQ_TFIDF_HT_POS_FT_hstack_path)
# NepalEQ_TFIDF_HT_POS_glove_hstack = np.load(NepalEQ_TFIDF_HT_POS_glove_hstack_path)
# NepalEQ_TFIDF_HT_POS_w2v_hstack = np.load(NepalEQ_TFIDF_HT_POS_w2v_hstack_path)
# NepalEQ_TFIDF_HT_POS_BERT_hstack = np.load(NepalEQ_TFIDF_HT_POS_BERT_hstack_path)


# # load targets
# NepalEQ_df_target = np.load(NepalEQ_df_target_path)
# # NepalEQ_target = np.load(NepalEQ_df_target)


# # df_list = [NepalEQ_hashtag, NepalEQ_POS, NepalEQ_df_embeddings_fasttext, NepalEQ_df_embeddings_word2vec,
# #            NepalEQ_df_embeddings_glove, NepalEQ_df_bert_input, NepalEQ_df_tfidf_1gram,
# #            NepalEQ_TFIDF_HT_hstack, NepalEQ_HT_POS_hstack, NepalEQ_HT_FT_hstack,
# #            NepalEQ_HT_glove_hstack, NepalEQ_HT_w2v_hstack, NepalEQ_HT_bert_hstack,
# #            NepalEQ_POS_FT_hstack, NepalEQ_POS_glove_hstack, NepalEQ_POS_w2v_hstack,
# #            NepalEQ_POS_bert_hstack, NepalEQ_TFIDF_POS_hstack, NepalEQ_TFIDF_FT_hstack,
# #            NepalEQ_TFIDF_glove_hstack, NepalEQ_TFIDF_w2v_hstack, NepalEQ_TFIDF_bert_hstack,
# #            NepalEQ_TFIDF_HT_POS_hstack, NepalEQ_TFIDF_HT_FT_hstack, NepalEQ_TFIDF_HT_glove_hstack, NepalEQ_TFIDF_HT_w2v_hstack]
# df_list = [NepalEQ_TFIDF_HT_bert_hstack, NepalEQ_TFIDF_HT_POS_FT_hstack, NepalEQ_TFIDF_HT_POS_glove_hstack,
#            NepalEQ_TFIDF_HT_POS_w2v_hstack, NepalEQ_TFIDF_HT_POS_BERT_hstack]
# # df_names = ['NepalEQ_hashtag', 'NepalEQ_POS', 'NepalEQ_df_embeddings_fasttext', 'NepalEQ_df_embeddings_word2vec',
# #            'NepalEQ_df_embeddings_glove', 'NepalEQ_df_bert_input', 'NepalEQ_df_tfidf_1gram',
# #            'NepalEQ_TFIDF_HT_hstack', 'NepalEQ_HT_POS_hstack', 'NepalEQ_HT_FT_hstack',
# #            'NepalEQ_HT_glove_hstack', 'NepalEQ_HT_w2v_hstack', 'NepalEQ_HT_bert_hstack',
# #            'NepalEQ_POS_FT_hstack', 'NepalEQ_POS_glove_hstack', 'NepalEQ_POS_w2v_hstack',
# #            'NepalEQ_POS_bert_hstack', 'NepalEQ_TFIDF_POS_hstack', 'NepalEQ_TFIDF_FT_hstack',
# #            'NepalEQ_TFIDF_glove_hstack', 'NepalEQ_TFIDF_w2v_hstack', 'NepalEQ_TFIDF_bert_hstack',
# #            'NepalEQ_TFIDF_HT_POS_hstack', 'NepalEQ_TFIDF_HT_FT_hstack', 'NepalEQ_TFIDF_HT_glove_hstack', 'NepalEQ_TFIDF_HT_w2v_hstack']
# df_names = ['NepalEQ_TFIDF_HT_bert_hstack', 'NepalEQ_TFIDF_HT_POS_FT_hstack', 'NepalEQ_TFIDF_HT_POS_glove_hstack',
#              'NepalEQ_TFIDF_HT_POS_w2v_hstack', 'NepalEQ_TFIDF_HT_POS_BERT_hstack']

In [4]:
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# y = NepalEQ_df_target


# for X, n in zip(df_list, df_names):
#   y = NepalEQ_df_target
#   # split into train test sets
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

#   # Scale the data
#   scaler = StandardScaler()
#   X_train = scaler.fit_transform(X_train)
#   X_test = scaler.transform(X_test)

#   # Create an instance of LinearSVC
#   svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

#   # Train the SVM on the training data
#   svm.fit(X_train, y_train)

#   # Compute the metrics
#   y_pred = svm.predict(X_test)
#   accuracy = accuracy_score(y_test, y_pred)
#   precision = precision_score(y_test, y_pred)
#   recall = recall_score(y_test, y_pred)
#   f1 = f1_score(y_test, y_pred)
#   auc = roc_auc_score(y_test, svm.decision_function(X_test))

#   # Create a DataFrame to store the metrics
#   metrics_df = pd.DataFrame({'name': [n], 'accuracy': [accuracy],
#                              'precision': [precision],
#                              'recall': [recall],
#                              'f1_score': [f1],'auc': [auc]})

#   print(metrics_df)

In [5]:
# vanilla SVM

from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

y = QueenslandFLD_target

metrics_list = []  # List to store the metrics for each iteration

for X, n in zip(df_list, df_names):
  y = QueenslandFLD_target
  # split into train test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

  # Scale the data
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # Create an instance of LinearSVC
  svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

  # Train the SVM on the training data
  svm.fit(X_train, y_train)

  # Compute the metrics
  y_pred = svm.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  auc = roc_auc_score(y_test, svm.decision_function(X_test))

  # Append the metrics to the list
  metrics_list.append({'name': n, 'accuracy': accuracy,
                       'precision': precision,
                       'recall': recall,
                       'f1_score': f1, 'auc': auc})

# Create the final dataframe
metrics_df = pd.DataFrame(metrics_list)
metrics_df


,name,accuracy,precision,recall,f1_score,auc
0,QueenslandFLD_hashtag,0.560354,0.551508,0.993846,0.709370,0.640290
1,QueenslandFLD_hashtag_embeddings,0.551495,0.547116,0.982564,0.702861,0.595769
2,QueenslandFLD_hashtag_tfidf,0.559247,0.551055,0.990769,0.708211,0.637525


In [ ]:
from sklearn.svm import SVC
# model = SVC(kernel='linear', random_state = 10)
# model.fit(x_train_dtm, y_train)
# #predicting output for test data
# pred = model.predict(x_test_dtm)


# vanilla SVM

from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

y = QueenslandFLD_target

metrics_list = []  # List to store the metrics for each iteration

for X, n in zip(df_list, df_names):
  y = QueenslandFLD_target
  # split into train test sets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

  # Scale the data
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_test = scaler.transform(X_test)

  # instantiate classifier with default hyperparameters with kernel=rbf, C=1.0 and gamma=auto
  model=SVC()



  # declare parameters for hyperparameter tuning
  parameters = [ {'C':[1, 10, 100, 1000], 'kernel':['linear']},
                {'C':[1, 10, 100, 1000], 'kernel':['rbf'], 'gamma':[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
                {'C':[1, 10, 100, 1000], 'kernel':['poly'], 'degree': [2,3,4] ,'gamma':[0.01,0.02,0.03,0.04,0.05]}
                ]




  grid_search = GridSearchCV(estimator = model,
                            param_grid = parameters,
                            scoring = 'accuracy',
                            cv = 5,
                            verbose=0)

  # Create an instance of LinearSVC
  # model = SVC(kernel='', random_state = 10)

  # Train the SVM on the training data
  grid_search.fit(X_train, y_train)

  # Compute the metrics
  y_pred = grid_search.predict(X_test)
  accuracy = accuracy_score(y_test, y_pred)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)
  auc = roc_auc_score(y_test, grid_search.decision_function(X_test))

  # Append the metrics to the list
  metrics_list.append({'name': n, 'accuracy': accuracy,
                       'precision': precision,
                       'recall': recall,
                       'f1_score': f1, 'auc': auc})

# Create the final dataframe
metrics_df = pd.DataFrame(metrics_list)
metrics_df


In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

y = QueenslandFLD_target

metrics_list = []  # List to store the metrics for each iteration

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for X, n in zip(df_list, df_names):
    # Scale the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Create an instance of LinearSVC
    svm = LinearSVC(dual=False, max_iter=10000)

    # Define the parameter grid for grid search
    param_grid = {'C': [0.01, 0.1, 1.0], 'penalty': ['l1', 'l2']}

    # Perform grid search with stratified k-fold cross-validation
    grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=skf)
    grid_search.fit(X, y)

    # Get the best estimator from grid search
    best_svm = grid_search.best_estimator_

    # Perform cross-validation with the best estimator
    scores = cross_val_score(best_svm, X, y, cv=skf, scoring='roc_auc')

    # Compute the metrics using the best estimator
    y_pred = best_svm.predict(X)
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    auc = scores.mean()

    # Append the metrics to the list
    metrics_list.append({'name': n, 'accuracy': accuracy,
                         'precision': precision,
                         'recall': recall,
                         'f1_score': f1, 'auc': auc})

# Create the final dataframe
metrics_df = pd.DataFrame(metrics_list)
metrics_df


,name,accuracy,precision,recall,f1_score,auc
0,QueenslandFLD_hashtag,0.613391,0.582779,0.998153,0.735898,0.654247
1,QueenslandFLD_hashtag_embeddings,0.572520,0.558120,0.997845,0.715848,0.595966
2,QueenslandFLD_hashtag_tfidf,0.613557,0.582854,0.998461,0.736042,0.655372


In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

def preprocess_data(X):
    # Scale the data
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X

def evaluate_model(X, y, classifier, param_grid, cv):
    # Perform grid search with stratified k-fold cross-validation
    grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, cv=cv)
    grid_search.fit(X, y)

    # Get the best estimator from grid search
    best_classifier = grid_search.best_estimator_

    # Perform cross-validation with the best estimator
    scores = cross_val_score(best_classifier, X, y, cv=cv, scoring='roc_auc')

    # Compute the metrics using the best estimator
    y_pred = best_classifier.predict(X)
    accuracy = accuracy_score(y, y_pred)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)
    f1 = f1_score(y, y_pred)
    auc = scores.mean()

    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1, 'auc': auc}

# df_list = [df1, df2, df3]  # List of datasets
# df_names = ['dataset1', 'dataset2', 'dataset3']  # List of dataset names
y = QueenslandFLD_target

metrics_list = []  # List to store the metrics for each iteration

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for X, n in zip(df_list, df_names):
    # Preprocess the data
    X = preprocess_data(X)

    # Create an instance of LinearSVC
    svm = LinearSVC(dual=False, max_iter=10000)

    # Define the parameter grid for grid search
    param_grid = {'C': [0.01, 0.1, 1.0], 'penalty': ['l1', 'l2']}

    # Evaluate the model
    metrics = evaluate_model(X, y, svm, param_grid, skf)
    metrics['name'] = n

    # Append the metrics to the list
    metrics_list.append(metrics)

# Create the final dataframe
metrics_df = pd.DataFrame(metrics_list)
metrics_df


,accuracy,precision,recall,f1_score,auc,name
0,0.613391,0.582779,0.998153,0.735898,0.654247,QueenslandFLD_hashtag
1,0.572520,0.558120,0.997845,0.715848,0.595966,QueenslandFLD_hashtag_embeddings
2,0.613557,0.582854,0.998461,0.736042,0.655372,QueenslandFLD_hashtag_tfidf


In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

y = QueenslandFLD_target

metrics_list = []  # List to store the metrics for each iteration

for X, n in zip(df_list, df_names):
    y = QueenslandFLD_target
    # split into train test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Feature selection using Recursive Feature Elimination (RFE)
    estimator = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)
    selector = RFE(estimator, n_features_to_select=10)  # Select top 10 features
    X_train = selector.fit_transform(X_train, y_train)
    X_test = selector.transform(X_test)

    # Feature engineering using Principal Component Analysis (PCA)
    pca = PCA(n_components=10)  # Reduce to 10 principal components
    X_train = pca.fit_transform(X_train)
    X_test = pca.transform(X_test)

    # Create an instance of LinearSVC
    svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

    # Train the SVM on the training data
    svm.fit(X_train, y_train)

    # Compute the metrics
    y_pred = svm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, svm.decision_function(X_test))

    # Append the metrics to the list
    metrics_list.append({'name': n, 'accuracy': accuracy,
                        'precision': precision,
                        'recall': recall,
                        'f1_score': f1, 'auc': auc})

# Create the final dataframe
metrics_df = pd.DataFrame(metrics_list)
metrics_df


,name,accuracy,precision,recall,f1_score,auc
0,QueenslandFLD_hashtag,0.562569,0.974359,0.194872,0.324786,0.594559
1,QueenslandFLD_hashtag_embeddings,0.546512,0.544622,0.976410,0.699229,0.588284
2,QueenslandFLD_hashtag_tfidf,0.564784,0.970149,0.200000,0.331633,0.596035


In [ ]:
# # Save the dataframe as a CSV file
# metrics_df.to_csv('NepalEQ_SVM2_metrics.csv', index=False)

In [ ]:
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

y = QueenslandFLD_target

metrics_list = []  # List to store the metrics for each iteration

# Create an instance of SVC as the base estimator
base_estimator = SVC(probability=True)

for X, n in zip(df_list, df_names):
    # split into train test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Create an AdaBoostClassifier with the base estimator
    ada_boost = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')

    # Train the AdaBoostClassifier on the training data
    ada_boost.fit(X_train, y_train)

    # Compute the metrics
    y_pred = ada_boost.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, ada_boost.decision_function(X_test))

    # Append the metrics to the list
    metrics_list.append({'name': n, 'accuracy': accuracy,
                         'precision': precision,
                         'recall': recall,
                         'f1_score': f1, 'auc': auc})

    # Create the final dataframe
    metrics_df = pd.DataFrame(metrics_list)
    print(metrics_df)


In [ ]:
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import SVC
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# from joblib import Parallel, delayed
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# import pandas as pd

# y = QueenslandFLD_target

# metrics_list = []  # List to store the metrics for each iteration

# # Define a function for parallel data splitting
# def split_data(X, y):
#     return train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

# # Use Parallel and delayed functions for parallel data splitting
# results = Parallel(n_jobs=-1)(
#     delayed(split_data)(X, y) for X in df_list
# )

# # Unpack the results into separate lists
# X_train_list, X_test_list, y_train_list, y_test_list = zip(*results)

# # Define the preprocessing pipeline
# preprocessor = ColumnTransformer([
#     ('scaler', StandardScaler(), slice(X_train.shape[1]))
# ])

# for X_train, X_test, y_train, y_test, n in zip(X_train_list, X_test_list, y_train_list, y_test_list, df_names):
#     # Create an AdaBoostClassifier pipeline with preprocessing
#     ada_boost = Pipeline([
#         ('preprocessor', preprocessor),
#         ('classifier', AdaBoostClassifier(base_estimator=SVC(probability=True), algorithm='SAMME'))
#     ])

#     # Train the AdaBoostClassifier pipeline on the training data
#     ada_boost.fit(X_train, y_train)

#     # Apply the preprocessing pipeline to the test data
#     X_test_preprocessed = preprocessor.transform(X_test)

#     # Compute the metrics
#     y_pred = ada_boost.predict(X_test_preprocessed)
#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred)
#     auc = roc_auc_score(y_test, ada_boost.decision_function(X_test_preprocessed))

#     # Append the metrics to the list
#     metrics_list.append({'name': n, 'accuracy': accuracy,
#                          'precision': precision,
#                          'recall': recall,
#                          'f1_score': f1, 'auc': auc})

# # Create the final dataframe
# metrics_df = pd.DataFrame(metrics_list)
# print(metrics_df)


In [ ]:
# #Adaboost

# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# from sklearn.ensemble import AdaBoostClassifier  # Import AdaBoostClassifier
# import pandas as pd

# y = QueenslandFLD_target

# metrics_list = []  # List to store the metrics for each iteration

# for X, n in zip(df_list, df_names):
#   y = QueenslandFLD_target
#   # split into train test sets
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

#   # Scale the data
#   scaler = StandardScaler()
#   X_train = scaler.fit_transform(X_train)
#   X_test = scaler.transform(X_test)

#   # Create an instance of AdaBoostClassifier with LinearSVC as the base estimator
#   base_estimator = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=500)
#   ada_boost = AdaBoostClassifier(base_estimator=base_estimator, algorithm='SAMME')

#   # Train the AdaBoostClassifier on the training data
#   ada_boost.fit(X_train, y_train)

#   # Compute the metrics
#   y_pred = ada_boost.predict(X_test)
#   accuracy = accuracy_score(y_test, y_pred)
#   precision = precision_score(y_test, y_pred)
#   recall = recall_score(y_test, y_pred)
#   f1 = f1_score(y_test, y_pred)
#   auc = roc_auc_score(y_test, ada_boost.decision_function(X_test))

#   # Append the metrics to the list
#   metrics_list.append({'name': n, 'accuracy': accuracy,
#                        'precision': precision,
#                        'recall': recall,
#                        'f1_score': f1, 'auc': auc})

# # Create the final dataframe
# metrics_df = pd.DataFrame(metrics_list)
# metrics_df


In [ ]:
# Save the dataframe as a CSV file
metrics_df.to_csv('QueenslandFLD_ASVM_metrics.csv', index=False)

In [ ]:
## Bagging

from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

y = QueenslandFLD_target

metrics_list = []  # List to store the metrics for each iteration

for X, n in zip(df_list, df_names):
    y = QueenslandFLD_target
    # split into train test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Create an instance of LinearSVC
    svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=5)

    # Create a BaggingClassifier with the SVM as the base estimator
    bagging = BaggingClassifier(base_estimator=svm, n_estimators=3, random_state=42)

    # Train the bagging classifier on the training data
    bagging.fit(X_train, y_train)

    # Compute the metrics
    y_pred = bagging.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, bagging.decision_function(X_test))

    # Append the metrics to the list
    metrics_list.append({'name': n, 'accuracy': accuracy,
                       'precision': precision,
                       'recall': recall,
                       'f1_score': f1, 'auc': auc})

    # Create the final dataframe
    metrics_df = pd.DataFrame(metrics_list)
    print(metrics_df)


In [ ]:
# Save the dataframe as a CSV file
metrics_df.to_csv('QueenslandFLD_BSVM_metrics.csv', index=False)

In [ ]:
## Random Subspace

from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np

# Set the random seed for reproducibility
np.random.seed(42)

y = QueenslandFLD_target

metrics_list = []  # List to store the metrics for each iteration

for X, n in zip(df_list, df_names):
    y = QueenslandFLD_target
    # split into train test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)

    # Scale the data
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Create an instance of LinearSVC
    svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10)

    # Random Subspace Method
    n_features = X_train.shape[1]
    subset_size = int(np.sqrt(n_features))  # Adjust this value as desired

    # Randomly select a subset of features
    random_features = np.random.choice(range(n_features), size=subset_size, replace=False)
    X_train_subset = X_train[:, random_features]
    X_test_subset = X_test[:, random_features]

    # Train the SVM on the training data subset
    svm.fit(X_train_subset, y_train)

    # Compute the metrics
    y_pred = svm.predict(X_test_subset)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, svm.decision_function(X_test_subset))

    # Append the metrics to the list
    metrics_list.append({'name': n, 'accuracy': accuracy,
                       'precision': precision,
                       'recall': recall,
                       'f1_score': f1, 'auc': auc})

    # Create the final dataframe
    metrics_df = pd.DataFrame(metrics_list)
    print(metrics_df)

In [ ]:
# Save the dataframe as a CSV file
metrics_df.to_csv('QueenslandFLD_RSSVM_metrics.csv', index=False)

### Hashtags

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_hashtag
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.565217,0.624294,0.223684,0.329359,0.58979


In [ ]:
# import pandas as pd
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.model_selection import train_test_split, GridSearchCV


# # Split the dataset into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Set up a parameter grid for the SVM
# param_grid = {
#     'C': [0.01, 0.1, 1, 10, 100],
#     'penalty': ['l1', 'l2'],
#     'dual': [False],
#     'max_iter': [1000, 5000, 10000]
# }

# # Create an instance of LinearSVC
# svm = LinearSVC()

# # Perform grid search to find the best hyperparameters
# grid_search = GridSearchCV(svm, param_grid=param_grid, cv=5)
# grid_search.fit(X_train, y_train)

# # Print the best hyperparameters
# print("Best hyperparameters:", grid_search.best_params_)

# # Train the SVM on the training data with the best hyperparameters
# best_svm = grid_search.best_estimator_
# best_svm.fit(X_train, y_train)

# # Compute the metrics on the test data with the best SVM
# y_pred = best_svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, best_svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_hashtag
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.560908,0.551822,0.993846,0.70963,0.640342


### POS

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_POS
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.539614,0.537155,0.256073,0.346813,0.571822


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_POS
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.653931,0.668593,0.711795,0.689518,0.699272


### Fasttext

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_df_embeddings_fasttext
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.65942,0.652973,0.611336,0.631469,0.712084


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_df_embeddings_fasttext
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.859358,0.861585,0.881026,0.871197,0.926656


### Word2Vec

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_df_embeddings_word2vec
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.651208,0.644252,0.601215,0.62199,0.707199


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_df_embeddings_word2vec
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.850498,0.861538,0.861538,0.861538,0.922397


### Glove

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_df_embeddings_glove
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.659903,0.65368,0.611336,0.631799,0.71394


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_df_embeddings_glove
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

ValueError: ignored

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.858804,0.86,0.882051,0.870886,0.925587


### Bert

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_df_bert_input
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.655072,0.681217,0.521255,0.590596,0.700288


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_df_bert_input
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.616833,0.632864,0.691282,0.660784,0.663945


### TF-IDF

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_df_tfidf_1gram
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.719324,0.728909,0.65587,0.690464,0.770403


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_df_tfidf_1gram
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.952935,0.972399,0.939487,0.95566,0.978918


### NepalEQ_TFIDF_HT_hstack

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_TFIDF_HT_RT_POS_FT
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.712077,0.725806,0.637652,0.678879,0.776022


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_TFIDF_HT_hstack
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.955703,0.975558,0.941538,0.958246,0.982399


### NepalEQ_HT_POS_hstack

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_TFIDF_HT_RT_POS_w2v
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.714493,0.725825,0.645749,0.683449,0.777428


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_HT_POS_hstack
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.951827,0.970339,0.939487,0.954664,0.981074


### NepalEQ_HT_FT_hstack

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_TFIDF_HT_RT_POS_glove
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.716908,0.728929,0.647773,0.685959,0.776319


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_HT_FT_hstack
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.95515,0.974522,0.941538,0.957746,0.980707


### NepalEQ_HT_glove_hstack

#### Nepal

In [ ]:
# # split imbalanced dataset into train and test sets with stratification
# from collections import Counter
# from sklearn.datasets import make_classification
# from sklearn.model_selection import train_test_split
# # create dataset
# X = NepalEQ_TFIDF_HT_RT_POS_bert
# y = NepalEQ_df_target
# # split into train test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
# print(Counter(y_train))
# print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
# from sklearn import datasets
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.svm import LinearSVC
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# # Scale the data
# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Create an instance of LinearSVC
# svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# # Train the SVM on the training data
# svm.fit(X_train, y_train)

# # # Evaluate performance on testing set
# # y_pred = svm.predict(X_test)
# # accuracy = accuracy_score(y_test, y_pred)
# # print("Accuracy: {:.2f}%".format(accuracy * 100))

# # # Calculate and print precision
# # precision = precision_score(y_test, y_pred)
# # print("Precision: {:.2f}%".format(precision * 100))

# # # Calculate and print recall
# # recall = recall_score(y_test, y_pred)
# # print("Recall: {:.2f}%".format(recall * 100))

# # # Calculate and print F1 score
# # f1 = f1_score(y_test, y_pred)
# # print("F1 score: {:.2f}%".format(f1 * 100))

# # # Calculate and print F1 score
# # auc = roc_auc_score(y_test, y_pred)
# # print("auc score: {:.2f}%".format(f1 * 100))


# # Compute the metrics
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)
# auc = roc_auc_score(y_test, svm.decision_function(X_test))

# # Create a DataFrame to store the metrics
# metrics_df = pd.DataFrame({'accuracy': [accuracy],
#                            'precision': [precision],
#                            'recall': [recall],
#                            'f1_score': [f1],
#                            'auc': [auc]})

# metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.716425,0.726554,0.65081,0.686599,0.776305


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_HT_glove_hstack
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.953488,0.973433,0.939487,0.956159,0.979292


### HT + POS + TF-IDF

#### Nepal

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_TFIDF_HT_POS
y = NepalEQ_df_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({0: 2524, 1: 2305})
Counter({0: 1082, 1: 988})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.716425,0.733411,0.637652,0.682187,0.77737


#### Queensland

In [ ]:
# split imbalanced dataset into train and test sets with stratification
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
# create dataset
X = NepalEQ_TFIDF_HT_POS
y = NepalEQ_target
# split into train test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)
print(Counter(y_train))
print(Counter(y_test))

Counter({1: 2273, 0: 1940})
Counter({1: 975, 0: 831})


In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [ ]:
# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create an instance of LinearSVC
svm = LinearSVC(C=0.1, penalty='l1', dual=False, max_iter=10000)

# Train the SVM on the training data
svm.fit(X_train, y_train)

# # Evaluate performance on testing set
# y_pred = svm.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Accuracy: {:.2f}%".format(accuracy * 100))

# # Calculate and print precision
# precision = precision_score(y_test, y_pred)
# print("Precision: {:.2f}%".format(precision * 100))

# # Calculate and print recall
# recall = recall_score(y_test, y_pred)
# print("Recall: {:.2f}%".format(recall * 100))

# # Calculate and print F1 score
# f1 = f1_score(y_test, y_pred)
# print("F1 score: {:.2f}%".format(f1 * 100))

# # Calculate and print F1 score
# auc = roc_auc_score(y_test, y_pred)
# print("auc score: {:.2f}%".format(f1 * 100))


# Compute the metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, svm.decision_function(X_test))

# Create a DataFrame to store the metrics
metrics_df = pd.DataFrame({'accuracy': [accuracy],
                           'precision': [precision],
                           'recall': [recall],
                           'f1_score': [f1],
                           'auc': [auc]})

metrics_df

,accuracy,precision,recall,f1_score,auc
0,0.954042,0.973461,0.940513,0.956703,0.979229
